# Script Contents
### -Import Libraries

### -Import Data Part 1

### -Check Data Shape

### -Import Data Part 2

### -Combine Data

### -Confirm and Investigate Combined Data

### -Export Combined Data

# Import Libraries

In [6]:
# Import libraries

import pandas as pd
import numpy as np
import os

## 03. In a new notebook, import the orders_products_combined dataframe from the pickle file you just saved.

In [8]:
# Create a path for my pain project folder
path = r"C:\Users\jacks\Instacart Basket Analysis 081424"

In [9]:
# Import the orders_produrts_combined pickle file from Prepared Data
df_orders_products_combined = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

## 04. Check the shape of the imported dataframe (it should be the same as the one you exported—always check!).

In [11]:
# Check the imported df shape
df_orders_products_combined.shape

(32434489, 12)

#### The shape is the same as the exported dataframe

## 05. Determine a suitable way to combine the orders_products_combined dataframe with your products data set. Make sure you’re using your wrangled, cleaned, and deduped products data set stored in your “Prepared Data” folder from the previous Exercise’s task.

In [14]:
# First, I need to import the latest products data set from my Prepared Data folder
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = False)

In [15]:
# Second, I need to check out the dataframes to get an idea of the best way to combine
df_prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [22]:
df_prods.describe()

,Unnamed: 0,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000,49672.000000
mean,24852.005053,24850.349775,67.762442,11.728942,9.993282
std,14342.265579,14340.705287,38.315784,5.850779,453.615536
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,12432.750000,35.000000,7.000000,4.100000
50%,24851.500000,24850.500000,69.000000,13.000000,7.100000
75%,37272.250000,37268.250000,100.000000,17.000000,11.100000
max,49692.000000,49688.000000,134.000000,21.000000,99999.000000


In [13]:
df_orders_products_combined.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,0,0,2539329,1,1,2,8,7.0,196,1,0,both
1,0,0,2539329,1,1,2,8,7.0,14084,2,0,both
2,0,0,2539329,1,1,2,8,7.0,12427,3,0,both
3,0,0,2539329,1,1,2,8,7.0,26088,4,0,both
4,0,0,2539329,1,1,2,8,7.0,26405,5,0,both


In [14]:
# Both dataframes share a product_id column so I am going to inner merge them using product_id as the key
df_ords_prods_merge = df_orders_products_combined.merge(df_prods, on = 'product_id', indicator = 'exists')

#### In the previous command, I used the argument "indicator = 'exists'" because, at first, I tried using "indicator = True". This resulted in an error since df_orders_products_combined already has a column named '_merge' that acts as an indicator. I found the "indicator = 'exists'" solution here: https://stackoverflow.com/questions/48669316/valueerror-cannot-use-name-of-an-existing-column-for-indicator-column

## 06. Confirm the results of the merge using the merge flag.

In [17]:
# First, I want to see what my combined data looks like
df_ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,Unnamed: 0_y,product_name,aisle_id,department_id,prices,exists
0,0,0,2539329,1,1,2,8,7.0,196,1,0,both,195,Soda,77,7,9.0,both
1,0,0,2539329,1,1,2,8,7.0,14084,2,0,both,14084,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,0,0,2539329,1,1,2,8,7.0,12427,3,0,both,12427,Original Beef Jerky,23,19,4.4,both
3,0,0,2539329,1,1,2,8,7.0,26088,4,0,both,26089,Aged White Cheddar Popcorn,23,19,4.7,both
4,0,0,2539329,1,1,2,8,7.0,26405,5,0,both,26406,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


In [31]:
# Count the values in the 'exists' column
df_ords_prods_merge['exists'].value_counts()

exists
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

#### The combined dataframe only has 32,404,859 rows. This is less than the 32,434,489 rows in the df_orders_products_combined dataframe, which tells me there are orders that either didn't include a product id or included a product id not found in the product dataframe. 

In [34]:
# I want to see what happens when I left join the dataframes
df_ords_prods_merge_left = df_orders_products_combined.merge(df_prods, on = 'product_id', how = 'left', indicator = 'exists')

In [38]:
# Check the count of the 'exists' column
df_ords_prods_merge_left['exists'].value_counts()

exists
both          32404859
left_only        30200
right_only           0
Name: count, dtype: int64

#### I think this information is telling me tha there are rows in the df_orders_products_combined dataframe with product_ids that don't match up with any product_ids in the df_prods dataframe. 

In [40]:
# I want to describe the df and see if I notice anything unusual
df_ords_prods_merge_left.describe()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,aisle_id,department_id,prices
count,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.243506e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07
mean,1.709857e+06,1.709857e+06,1.710748e+06,1.029372e+05,1.714195e+01,2.738815e+00,1.342498e+01,1.084118e+01,2.557635e+04,8.351081e+00,5.896955e-01,2.560037e+04,7.119612e+01,9.919792e+00,1.198023e+01
std,9.864641e+05,9.864641e+05,9.873006e+05,5.946651e+04,1.753496e+01,2.090051e+00,4.246357e+00,8.552310e+00,1.409657e+04,7.126656e+00,4.918889e-01,1.408555e+04,3.821139e+01,6.281485e+00,4.956554e+02
min,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,8.556120e+05,8.556120e+05,8.559430e+05,5.142000e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353200e+04,3.000000e+00,0.000000e+00,1.354400e+04,3.100000e+01,4.000000e+00,4.200000e+00
50%,1.709136e+06,1.709136e+06,1.711047e+06,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00,2.525600e+04,6.000000e+00,1.000000e+00,2.530300e+04,8.300000e+01,9.000000e+00,7.400000e+00
75%,2.565949e+06,2.565949e+06,2.565514e+06,1.543910e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.793500e+04,1.100000e+01,1.000000e+00,3.795100e+04,1.070000e+02,1.600000e+01,1.130000e+01
max,3.421081e+06,3.421081e+06,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,4.969200e+04,1.340000e+02,2.100000e+01,9.999900e+04


In [42]:
df_ords_prods_merge.describe()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,aisle_id,department_id,prices
count,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07
mean,1.709858e+06,1.709858e+06,1.710745e+06,1.029372e+05,1.714230e+01,2.738867e+00,1.342515e+01,1.084114e+01,2.559866e+04,8.352547e+00,5.895873e-01,2.560037e+04,7.119612e+01,9.919792e+00,1.198023e+01
std,9.864575e+05,9.864575e+05,9.872988e+05,5.946610e+04,1.753532e+01,2.090077e+00,4.246380e+00,8.552435e+00,1.408400e+04,7.127071e+00,4.919087e-01,1.408555e+04,3.821139e+01,6.281485e+00,4.956554e+02
min,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,8.556370e+05,8.556370e+05,8.559470e+05,5.142200e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.354400e+04,3.000000e+00,0.000000e+00,1.354400e+04,3.100000e+01,4.000000e+00,4.200000e+00
50%,1.709176e+06,1.709176e+06,1.711049e+06,1.026160e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00,2.530200e+04,6.000000e+00,1.000000e+00,2.530300e+04,8.300000e+01,9.000000e+00,7.400000e+00
75%,2.565919e+06,2.565919e+06,2.565499e+06,1.543890e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.794700e+04,1.100000e+01,1.000000e+00,3.795100e+04,1.070000e+02,1.600000e+01,1.130000e+01
max,3.421081e+06,3.421081e+06,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,4.969200e+04,1.340000e+02,2.100000e+01,9.999900e+04


In [46]:
# Check the number of null values in the left joined df
df_ords_prods_merge_left.isnull().sum()

Unnamed: 0.1                  0
Unnamed: 0_x                  0
order_id                      0
user_id                       0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_prior_order        0
product_id                    0
add_to_cart_order             0
reordered                     0
_merge                        0
Unnamed: 0_y              30200
product_name              30200
aisle_id                  30200
department_id             30200
prices                    30200
exists                        0
dtype: int64

#### At this point, I think the most reasonable solution is to export the inner joined dataframe so as to not export a dataframe with many null values. Although I will be excluding the rows from the df_orders_prodcts_combined dataframe by inner joining with the df_prods dataframe, I don't see the benefit of continuing my analysis with order data that doesn't match up with specific product data. 

In [51]:
# Before exporting, I want to drop the unnamed and unnecessary columns from the dataframe
df_ords_prods_merge = df_ords_prods_merge.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y', '_merge'])

In [53]:
# Check the results of the drop
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,exists
0,2539329,1,1,2,8,7.0,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,7.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,7.0,12427,3,0,Original Beef Jerky,23,19,4.4,both
3,2539329,1,1,2,8,7.0,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both
4,2539329,1,1,2,8,7.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


## 07. Export the newly created dataframe as ords_prods_merge in a suitable format (taking into consideration the size).

In [56]:
# Considering the size of the df, I am going to export as a pickle file
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))